In [1]:
options(warn=-1)
gc()
folder <- "analysis1201"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,509150,27.2,1095217,58.5,619098,33.1
Vcells,974209,7.5,8388608,64.0,1606620,12.3


In [2]:
library(xgboost)
library(readr)
library(stringr)
library(caret)
library(car)
library(DMwR)
library(Hmisc)
library(plyr)
library(dplyr)
library(tidyr)
library(data.table)
library(DescTools)
library(Matrix)
library(glmnet)
library(plotrix)
library(Ckmeans.1d.dp)

set.seed(0)
setwd("C:/Users/Jeane/MSDS/Proj2")

Loading required package: lattice
Loading required package: ggplot2
Loading required package: carData
Loading required package: grid
Loading required package: survival

Attaching package: 'survival'

The following object is masked from 'package:caret':

    cluster

Loading required package: Formula

Attaching package: 'Hmisc'

The following objects are masked from 'package:base':

    format.pval, units


Attaching package: 'plyr'

The following objects are masked from 'package:Hmisc':

    is.discrete, summarize

The following object is masked from 'package:DMwR':

    join


Attaching package: 'dplyr'

The following objects are masked from 'package:plyr':

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize

The following objects are masked from 'package:Hmisc':

    src, summarize

The following object is masked from 'package:car':

    recode

The following object is masked from 'package:xgboost':

    slice

The following objects are masked from 'pack

In [3]:
X_all_feature <- readRDS(paste0("cache/",folder,"/X_all_feature.RData"))
X_id <- readRDS(paste0("cache/",folder,"/X_id.RData"))
X_test_id <- readRDS(paste0("cache/",folder,"/X_test_id.RData"))
X_train_id <- readRDS(paste0("cache/",folder,"/X_train_id.RData"))
X_valid_id <- readRDS(paste0("cache/",folder,"/X_valid_id.RData"))

y <- readRDS(paste0("cache/",folder,"/y.RData"))
y_test <- readRDS(paste0("cache/",folder,"/y_test.RData"))
y_train <- readRDS(paste0("cache/",folder,"/y_train.RData"))
y_valid <- readRDS(paste0("cache/",folder,"/y_valid.RData"))

X_sp <- readRDS(paste0("cache/",folder,"/X_sp.RData"))
X_test_sp <- readRDS(paste0("cache/",folder,"/X_test_sp.RData"))
X_train_sp <- readRDS(paste0("cache/",folder,"/X_train_sp.RData"))
X_valid_sp <- readRDS(paste0("cache/",folder,"/X_valid_sp.RData"))

dX <- xgb.DMatrix(paste0("cache/",folder,"/dX.xgb.DMatrix.data"))
dX_test <- xgb.DMatrix(paste0("cache/",folder,"/dX_test.xgb.DMatrix.data"))
dX_train <- xgb.DMatrix(paste0("cache/",folder,"/dX_train.xgb.DMatrix.data"))
dX_test <- xgb.DMatrix(paste0("cache/",folder,"/dX_test.xgb.DMatrix.data"))


[16:24:09] 213451x269 matrix with 57418319 entries loaded from cache/analysis1201/dX.xgb.DMatrix.data
[16:24:09] 62096x269 matrix with 16703824 entries loaded from cache/analysis1201/dX_test.xgb.DMatrix.data
[16:24:11] 193116x269 matrix with 51948204 entries loaded from cache/analysis1201/dX_train.xgb.DMatrix.data
[16:24:11] 62096x269 matrix with 16703824 entries loaded from cache/analysis1201/dX_test.xgb.DMatrix.data


# Make the Normalized Discounted Cummulative Gains evaluation metric

In [4]:
dcg_at_k <- function (r, k=min(5, length(r)) ) {
  #only coded alternative formulation of DCG (used by kaggle)
  r <- as.vector(r)[1:k]
  sum(( 2^r - 1 )/ log2( 2:(length(r)+1)) )
} 

ndcg_at_k <- function(r, k=min(5, length(r)) ) {
  r <- as.vector(r)[1:k]
  if (sum(r) <= 0) return (0)     # no hits (dcg_max = 0)
  dcg_max = dcg_at_k(sort(r, decreasing=TRUE)[1:k], k)
  return ( dcg_at_k(r, k) / dcg_max )
}

score_predictions <- function(preds, truth) {
  # preds: matrix or data.frame
  # one row for each observation, one column for each prediction.
  # Columns are sorted from left to right descending in order of likelihood.
  # truth: vector
  # one row for each observation.
  preds <- as.matrix(preds)
  truth <- as.vector(truth)
  
  stopifnot( length(truth) == nrow(preds))
  r <- apply( cbind( truth, preds), 1
              , function(x) ifelse( x == x[1], 1, 0))[ -1, ]
  if ( ncol(preds) == 1) r <-  rbind( r, r)  #workaround for 1d matrices
  as.vector( apply(r, 2, ndcg_at_k) )
}

ndcg5 <- function(preds, dtrain) {
  
  labels <- getinfo(dtrain,"label")
  num.class = length(unique(labels))
  pred <- matrix(preds, nrow = num.class)
  top <- t(apply(pred, 2, function(y) order(y)[num.class:(num.class-4)]-1))
  
  x <- ifelse(top==labels,1,0)
  dcg <- function(y) sum((2^y - 1)/log(2:(length(y)+1), base = 2))
  ndcg <- mean(apply(x,1,dcg))
  return(list(metric = "ndcg5", value = ndcg))
}

In [5]:
Mean <- function(x) mean(x, na.rm=TRUE)
Median <- function(x) median(x, na.rm=TRUE)
Sd <- function(x) sd(x, na.rm=TRUE)
Sum <- function(x) sum(x, na.rm=TRUE)
Max <- function(x) max(x, na.rm=TRUE)
Min <- function(x) min(x, na.rm=TRUE)
Mean_value <- function(x) ifelse(is.nan(mean(x, na.rm=TRUE))==T, NA, mean(x, na.rm=TRUE))
Sum_value <- function(x) ifelse(sum(!is.na(x))==0, NA, sum(x, na.rm=TRUE))
Max_value <- function(x) ifelse(is.infinite(max(x, na.rm=TRUE))==T, NA, max(x, na.rm=TRUE))
Min_value <- function(x) ifelse(is.infinite(min(x, na.rm=TRUE))==T, NA, min(x, na.rm=TRUE))


# Hyperparatmenters Exploration

### ETA

In [7]:
#Eta 0.05
param2 <- list("objective" = "multi:softprob",
                "num_class" = 12,
                "eta" = 0.05,
                "max_depth" = 6,
                "subsample" = 0.5,
                "colsample_bytree" = 0.3,
                "alpha" = 1.0,
                "nthread" = 24)
cv_model = xgb.cv(param = param2,
                  data = dX_train, 
                  nfold = 10,
                  nrounds = 10,
                  maximize = TRUE,
                  feval = ndcg5,
                  early.stop.round = 10
  )

[1]	train-ndcg5:0.861170+0.037563	test-ndcg5:0.860050+0.038637 
Multiple eval metrics are present. Will use test_ndcg5 for early stopping.
Will train until test_ndcg5 hasn't improved in 10 rounds.



In [9]:
#Eta 0.1
param3 <- list("objective" = "multi:softprob",
                "num_class" = 12,
                "eta" = 0.1,
                "max_depth" = 6,
                "subsample" = 0.5,
                "colsample_bytree" = 0.3,
                "alpha" = 1.0,
                "nthread" = 24)
cv_model3 = xgb.cv(param = param3,
                  data = dX_train, 
                  nfold = 2,
                  nrounds = 1,
                  maximize = TRUE,
                  feval = ndcg5,
                  early.stop.round = 10
  )

[1]	train-ndcg5:0.841801+0.029293	test-ndcg5:0.840173+0.028222 
Multiple eval metrics are present. Will use test_ndcg5 for early stopping.
Will train until test_ndcg5 hasn't improved in 10 rounds.



### Max Depth

In [12]:
#MD 3
param4 <- list("objective" = "multi:softprob",
                "num_class" = 12,
                "eta" = 0.05,
                "max_depth" = 3,
                "subsample" = 0.5,
                "colsample_bytree" = 0.3,
                "alpha" = 1.0,
                "nthread" = 24)
cv_model4 = xgb.cv(param = param4,
                  data = dX_train, 
                  nfold = 10,
                  nrounds = 10,
                  maximize = TRUE,
                  feval = ndcg5,
                  early.stop.round = 10)

[1]	train-ndcg5:0.855574+0.039057	test-ndcg5:0.855206+0.039286 
Multiple eval metrics are present. Will use test_ndcg5 for early stopping.
Will train until test_ndcg5 hasn't improved in 10 rounds.



In [13]:
#MD 6
param5 <- list("objective" = "multi:softprob",
                "num_class" = 12,
                "eta" = 0.05,
                "max_depth" = 6,
                "subsample" = 0.5,
                "colsample_bytree" = 0.3,
                "alpha" = 1.0,
                "nthread" = 24)
cv_model5 = xgb.cv(param = param5,
                  data = dX_train, 
                  nfold = 10,
                  nrounds = 10,
                  maximize = TRUE,
                  feval = ndcg5,
                  early.stop.round = 10)

[1]	train-ndcg5:0.895707+0.000241	test-ndcg5:0.894410+0.000578 
Multiple eval metrics are present. Will use test_ndcg5 for early stopping.
Will train until test_ndcg5 hasn't improved in 10 rounds.



In [14]:
#MD 12
param5 <- list("objective" = "multi:softprob",
                "num_class" = 12,
                "eta" = 0.05,
                "max_depth" = 12,
                "subsample" = 0.5,
                "colsample_bytree" = 0.3,
                "alpha" = 1.0,
                "nthread" = 24)
cv_model5 = xgb.cv(param = param5,
                  data = dX_train, 
                  nfold = 10,
                  nrounds = 10,
                  maximize = TRUE,
                  feval = ndcg5,
                  early.stop.round = 10)

[1]	train-ndcg5:0.883576+0.010525	test-ndcg5:0.877263+0.011081 
Multiple eval metrics are present. Will use test_ndcg5 for early stopping.
Will train until test_ndcg5 hasn't improved in 10 rounds.



#### xgb.cv is a cross-validation feature in xgboost to estimate error
#### https://rpubs.com/mharris/multiclass_xgboost

In [25]:
nDCG_score_df <- data.frame()
for(i in 1:10){
  i <- 1
  start_time <- proc.time()
  set.seed(123 * i)
  rate <- 0.9
  
  feature_set_index <- createDataPartition(1:ncol(X_sp), p = rate,
                                           list = FALSE,
                                           times = 1)
  feature_set_index <- feature_set_index[,1]
  X_sp_ <- X_sp[, feature_set_index]
  X_test_sp_ <- X_test_sp[, feature_set_index]
  X_all_feature_ <- subset(X_all_feature, feature %in% feature_set_index)
  
  dX_ <- xgb.DMatrix(X_sp_, label = y, missing = -99999)
  
  param <- list("objective" = "multi:softprob",
                "num_class" = 12,
                "eta" = 0.05,
                "max_depth" = 6,
                "subsample" = 0.5,
                "colsample_bytree" = 0.3,
                "alpha" = 1.0,
                "nthread" = 24)
  
  # Run Cross Valication
  cv.nround = 2
  #cv.nround = 30
  bst.cv = xgb.cv(param = param,
                  data = dX_, 
                  nfold = 5,
                  nrounds = cv.nround,
                  maximize = TRUE,
                  feval = ndcg5,
                  early.stop.round = 10
  )

  elapse_time <- proc.time() - start_time
  nDCG_score_df_ <- data.frame(
    feature_set = i,
    nDCG = max(bst.cv$evaluation_log$test_ndcg5_mean),
    elapse_time = elapse_time[[3]],
    n = which.max(bst.cv$evaluation_log$test_ndcg5_mean)
  )
  nDCG_score_df <- bind_rows(
    nDCG_score_df,
    nDCG_score_df_
  )
  saveRDS(X_all_feature_, paste0("cache/",folder,"/test/X_all_feature",i,".RData"))
  saveRDS(nDCG_score_df, paste0("cache/",folder,"/test/nDCG_score_df.RData"))
}


[1]	train-ndcg5:0.852630+0.026891	test-ndcg5:0.851593+0.026451 
Multiple eval metrics are present. Will use test_ndcg5 for early stopping.
Will train until test_ndcg5 hasn't improved in 10 rounds.

[2]	train-ndcg5:0.857707+0.022058	test-ndcg5:0.856642+0.021962 
[1]	train-ndcg5:0.852630+0.026891	test-ndcg5:0.851593+0.026451 
Multiple eval metrics are present. Will use test_ndcg5 for early stopping.
Will train until test_ndcg5 hasn't improved in 10 rounds.

[2]	train-ndcg5:0.857707+0.022058	test-ndcg5:0.856642+0.021962 
[1]	train-ndcg5:0.852630+0.026891	test-ndcg5:0.851593+0.026451 
Multiple eval metrics are present. Will use test_ndcg5 for early stopping.
Will train until test_ndcg5 hasn't improved in 10 rounds.

[2]	train-ndcg5:0.857707+0.022058	test-ndcg5:0.856642+0.021962 
[1]	train-ndcg5:0.852630+0.026891	test-ndcg5:0.851593+0.026451 
Multiple eval metrics are present. Will use test_ndcg5 for early stopping.
Will train until test_ndcg5 hasn't improved in 10 rounds.

[2]	train-ndcg5:

In [43]:
nDCG_score_df <- readRDS(paste0("cache/",folder,"/test/nDCG_score_df.RData"))

i <- which.max(nDCG_score_df$nDCG)
n <- nDCG_score_df$n[which.max(nDCG_score_df$n)]
X_all_feature <- readRDS(paste0("cache/",folder,"/X_all_feature.RData"))
X_all_feature_ <- readRDS(paste0("cache/",folder,"/test/X_all_feature",i,".RData"))

In [40]:
nDCG_score_df <- readRDS(paste0("cache/",folder,"/test/nDCG_score_df.RData"))

i <- which.max(nDCG_score_df$nDCG)
n <- nDCG_score_df$n[which.max(nDCG_score_df$n)]
X_all_feature <- readRDS(paste0("cache/",folder,"/X_all_feature.RData"))
X_all_feature_ <- readRDS(paste0("cache/",folder,"/test/X_all_feature",
                                 i,".RData"))

X_all_feature_select <- subset(X_all_feature,
                               feature_name %in% X_all_feature_$feature_name)

# Final Training

In [49]:
dX_test_ <- xgb.DMatrix(X_test_sp_, missing = -99999)
    
    xgb <- xgboost(data = dX_, 
                   eta = 0.05,
                   max_depth = 6,  
                   nround = 100, 
                   subsample = 0.5,
                   colsample_bytree = 0.3,
                   alpha = 1.0,
                   eval_metric = ndcg5,
                   prediction = TRUE,
                   maximize = TRUE,
                   objective = "multi:softprob",
                   num_class = 12,
                   nthread = 24)

[1]	train-ndcg5:0.823590 
[2]	train-ndcg5:0.864986 
[3]	train-ndcg5:0.874783 
[4]	train-ndcg5:0.868293 
[5]	train-ndcg5:0.873097 
[6]	train-ndcg5:0.885656 
[7]	train-ndcg5:0.875659 
[8]	train-ndcg5:0.873228 
[9]	train-ndcg5:0.877974 
[10]	train-ndcg5:0.886876 
[11]	train-ndcg5:0.886172 
[12]	train-ndcg5:0.886695 
[13]	train-ndcg5:0.886213 
[14]	train-ndcg5:0.887108 
[15]	train-ndcg5:0.888606 
[16]	train-ndcg5:0.889093 
[17]	train-ndcg5:0.891906 
[18]	train-ndcg5:0.892788 
[19]	train-ndcg5:0.892889 
[20]	train-ndcg5:0.892643 
[21]	train-ndcg5:0.892975 
[22]	train-ndcg5:0.892797 
[23]	train-ndcg5:0.893186 
[24]	train-ndcg5:0.893271 
[25]	train-ndcg5:0.893313 
[26]	train-ndcg5:0.893342 
[27]	train-ndcg5:0.893212 
[28]	train-ndcg5:0.893332 
[29]	train-ndcg5:0.893414 
[30]	train-ndcg5:0.893508 
[31]	train-ndcg5:0.893591 
[32]	train-ndcg5:0.893599 
[33]	train-ndcg5:0.893795 
[34]	train-ndcg5:0.893984 
[35]	train-ndcg5:0.894166 
[36]	train-ndcg5:0.894266 
[37]	train-ndcg5:0.894552 
[38]	train

In [ ]:
X_all_imp <- xgb.importance(X_all_feature$feature_name, model=xgb)
saveRDS(X_all_imp, paste0("cache/",folder,"test/X_all_imp",i,".RData"))

In [52]:
y_pred <- predict(xgb, dX_test_)
    

In [53]:
predictions <- as.data.frame(t(matrix(y_pred, nrow=12)))
colnames(predictions) <- c('NDF','US','other','FR','CA','GB','ES','IT','PT','NL','DE','AU')

X_test_predictions <- bind_cols(
  X_test_id,
  predictions 
)
saveRDS(X_test_predictions, paste0("cache/",folder,"/test/X_test_predictions_part2_",i,".RData"))

In [ ]:
test_prediction <- matrix(test_pred, nrow = numberOfClasses,
                          ncol=length(test_pred)/numberOfClasses) %>%
  t() %>%
  data.frame() %>%
  mutate(label = test_label + 1,
         max_prob = max.col(., "last"))
# confusion matrix of test set
confusionMatrix(factor(test_prediction$max_prob),
                factor(test_prediction$label),
                mode = "everything")

In [63]:
#names <-  colnames(X_sp[,-1])
# compute feature importance matrix
importance_matrix = xgb.importance( model = xgb)

head(importance_matrix)


Feature,Gain,Cover,Frequency
dfb_tfa_lag,0.41723141,0.06125200,0.027901496
dfb_dac_lag,0.33088981,0.06775236,0.036667845
action_type_flg_sum_booking_request,0.04441529,0.04230923,0.010250972
population_in_thousands.AU,0.02120187,0.02862204,0.020219159
age_pred,0.01510186,0.06464245,0.057523271
signup_method_facebook,0.01082172,0.01517188,0.006621892


In [67]:
saveRDS(importance_matrix, 'importance_matrix.RData' )

In [55]:
#head(X_test_predictions)

id,id_num,country_destination,country_destination_num,NDF,US,other,FR,CA,GB,ES,IT,PT,NL,DE,AU
0010k6l0om,5,NA,NA,0.9184868,0.03087600,0.011163361,0.007745469,0.003995106,0.004744736,0.005490213,0.004908819,0.002677951,0.003515013,0.003351222,0.003045295
0031awlkjq,15,NA,NA,0.9224661,0.02995232,0.008223944,0.008339519,0.003578790,0.004995737,0.005411032,0.004625581,0.002441026,0.003750966,0.003247468,0.002967548
00378ocvlh,17,NA,NA,0.7292290,0.16250139,0.027847139,0.018173544,0.008699981,0.010556992,0.011082540,0.009261982,0.004798527,0.006465542,0.005970321,0.005413023
0048rkdgb1,24,NA,NA,0.9601074,0.01236672,0.005195642,0.003204134,0.002458523,0.002699940,0.002759703,0.002711839,0.001859983,0.002266153,0.002270945,0.002099045
0057snrdpu,26,NA,NA,0.9001185,0.04500182,0.012842843,0.008533031,0.004529670,0.005127040,0.005242714,0.004882999,0.002763306,0.003748629,0.004007328,0.003202155
005v5uf4dh,28,NA,NA,0.9587571,0.01312525,0.005802835,0.003313197,0.002383424,0.002682557,0.002777684,0.002726512,0.001820496,0.002230286,0.002241236,0.002139409


In [68]:
saveRDS(X_test_predictions, 'X_test_predictions.RData')

In [56]:
summary(xgb)

               Length  Class              Mode       
handle               1 xgb.Booster.handle externalptr
raw            3281283 -none-             raw        
niter                1 -none-             numeric    
evaluation_log       2 data.table         list       
call                23 -none-             call       
params              10 -none-             list       
callbacks            2 -none-             list       
feature_names      245 -none-             character  
nfeatures            1 -none-             numeric    

In [71]:
plot(xgb$evaluation_log)


ERROR: Error in png(tf, width, height, "in", pointsize, bg, res, antialias = antialias): unable to start png() device


plot without title

## png error in jupyter, pending solution. Switching to R Studio markdown files
###  https://stackoverflow.com/questions/50944878/r-graphs-in-jupyter-unable-to-start-png-device?newreg=2965792ff0954d63b02cc005701a8c1a

In [59]:
xgb.save(xgb, '1201xgb.model')

[1] TRUE

In [61]:
saveRDS(xgb$evaluation_log, 'elog.RData' )

In [69]:
save.image(file = "trn_wrkspc.RData")